<a href="https://colab.research.google.com/github/BlackPavvn/CAEX-Challenge/blob/develop/Proyecto_Reto_UTEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Predicción de mantenimiento en camiones CAEX

In [2]:
!apt-get install openjdk-11-jdk -qq > /dev/null

In [6]:
!wget https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

--2025-07-07 17:27:19--  https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388341449 (370M) [application/x-gzip]
Saving to: ‘spark-3.4.1-bin-hadoop3.tgz’

spark-3.4.1-bin-had 100%[===================>] 370.35M  8.23MB/s    in 37s     

2025-07-07 17:27:57 (9.94 MB/s) - ‘spark-3.4.1-bin-hadoop3.tgz’ saved [388341449/388341449]



In [7]:
!tar xf spark-3.4.1-bin-hadoop3.tgz

In [8]:
!pip install -q findspark

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetoUTEC") \
    .getOrCreate()

In [18]:
test_1 = spark.read.option("header", True).csv(f"/content/alertas.csv", inferSchema=True)
test_2 = spark.read.option("header", True).csv(f"/content/manual_motor.csv", inferSchema=True)

test_1.printSchema()
print(test_2.count())

root
 |-- _id: string (nullable = true)
 |-- awhen: timestamp (nullable = true)
 |-- idfalla: integer (nullable = true)
 |-- serie: string (nullable = true)
 |-- modelo: string (nullable = true)
 |-- tipo: integer (nullable = true)
 |-- codigo: string (nullable = true)
 |-- descripcion: string (nullable = true)
 |-- fecha1: timestamp (nullable = true)
 |-- valor1: double (nullable = true)
 |-- fecha2: timestamp (nullable = true)
 |-- valor2: double (nullable = true)
 |-- cont: integer (nullable = true)
 |-- fecharecepcion: timestamp (nullable = true)

197


In [17]:
test_3 = spark.read.option("header", True).csv(f"/content/mnp_caex_v1.csv", inferSchema=True)
test_3.show(3,0)

+------------------------+-----------------------+----------+------------+-------------------------------------------+---------+-----+--------------+----------+---------+-------------------+-------------------+-------+----------+-------+----------------------------------------+------------+------------------------+
|_id                     |awhen                  |status    |statussymbol|reason                                     |equipment|fleet|equipment_type|location  |shiftname|startlocal         |endlocal           |crew   |pay_number|id     |comment                                 |symbol      |name                    |
+------------------------+-----------------------+----------+------------+-------------------------------------------+---------+-----+--------------+----------+---------+-------------------+-------------------+-------+----------+-------+----------------------------------------+------------+------------------------+
|61f44383a2b9f391b446ff03|2022-03-08 01:23:57.079